In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/soil-types/Soil types/Yellow Soil/20.jpg
/kaggle/input/soil-types/Soil types/Yellow Soil/6.jpg
/kaggle/input/soil-types/Soil types/Yellow Soil/5.jpg
/kaggle/input/soil-types/Soil types/Yellow Soil/8.jpg
/kaggle/input/soil-types/Soil types/Yellow Soil/30.jpg
/kaggle/input/soil-types/Soil types/Yellow Soil/10.jpg
/kaggle/input/soil-types/Soil types/Yellow Soil/9.jpg
/kaggle/input/soil-types/Soil types/Yellow Soil/29.jpg
/kaggle/input/soil-types/Soil types/Yellow Soil/16.jpg
/kaggle/input/soil-types/Soil types/Yellow Soil/23.jpg
/kaggle/input/soil-types/Soil types/Yellow Soil/7.jpg
/kaggle/input/soil-types/Soil types/Yellow Soil/28.jpg
/kaggle/input/soil-types/Soil types/Yellow Soil/22.jpg
/kaggle/input/soil-types/Soil types/Yellow Soil/24.jpg
/kaggle/input/soil-types/Soil types/Yellow Soil/13.jpg
/kaggle/input/soil-types/Soil types/Yellow Soil/17.jpg
/kaggle/input/soil-types/Soil types/Yellow Soil/26.jpg
/kaggle/input/soil-types/Soil types/Yellow Soil/15.jpg
/kaggle/input/s

In [2]:
import tensorflow as tf
from tensorflow import keras

In [3]:
image_size = 220
batch_size = 20


target_size = (image_size, image_size)
input_shape = (image_size, image_size, 3)

# epochs = 20

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255)


train_generator = train_datagen.flow_from_directory(
        '../input/soil-types/Soil types/', 
        target_size=(200, 200),
        batch_size = batch_size,
        
        classes = ['Peat Soil','Cinder Soil', 'Black Soil', 'Laterite Soil', 'Yellow Soil'],
       
        class_mode='categorical')

Found 156 images belonging to 5 classes.


In [5]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 200x 200 with 3 bytes color
    # The first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(200, 200, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fifth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a dense layer
    tf.keras.layers.Flatten(),
    # 128 neuron in the fully-connected layer
    tf.keras.layers.Dense(128, activation='relu'),
    # 5 output neurons for 5 classes with the softmax activation
    tf.keras.layers.Dense(5, activation='softmax')
])

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 198, 198, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 99, 99, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 97, 97, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 48, 48, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 46, 46, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 23, 23, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 21, 21, 64)        3

In [7]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(learning_rate=0.001),
              metrics=['acc'])

In [8]:
total_sample=train_generator.n

In [9]:
n_epochs = 3

In [10]:
history = model.fit(
        train_generator, 
        steps_per_epoch=int(total_sample/batch_size),  
        epochs=n_epochs,
        verbose=1)

Epoch 1/3
7/7 [==============================] - 4s 363ms/step - loss: 1.6496 - acc: 0.1729
Epoch 2/3
7/7 [==============================] - 3s 355ms/step - loss: 1.5593 - acc: 0.3317
Epoch 3/3
7/7 [==============================] - 3s 359ms/step - loss: 1.3981 - acc: 0.3618


In [11]:
model.save(filepath="save_model/")

In [12]:
converter = tf.lite.TFLiteConverter.from_saved_model('save_model')
tflite_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_model)

921292